In [1]:
%matplotlib inline
import os, glob, sys
import pandas as pd
import numpy as np
import datetime as dt
from math import sqrt
from collections import Counter
import copy

In [2]:
allfiles = glob.glob(os.path.join('..','data\competition_data','*.csv'))
comp_files = glob.glob(os.path.join('..','data\competition_data','comp_*'))
newtrain_df = pd.read_csv(os.path.join('..','train.csv'),index_col = 0)

all_files = {}
for afile in allfiles:
    key = afile.split('\\')[-1].split('.')[0]
    all_files[key] = pd.read_csv(afile, header = 0)
all_files['train_set'] = pd.read_csv('../data/competition_data/train_set.csv', header = 0, parse_dates = ['quote_date'])
all_files['test_set'] = pd.read_csv('../data/competition_data/test_set.csv', header = 0, parse_dates = ['quote_date'])

# Components
comp_dict = {}
for compfile in comp_files:
    key = compfile.split('\\')[-1].split('.')[0]
    comp_dict[key] = pd.read_csv(compfile,header=0, index_col=0)

# The rest of the files
rest = ['components.csv','specs.csv','tube_end_form.csv','type_component.csv','type_connection.csv','type_end_form.csv']
restfile = []
for x in rest:
    restfile += [os.path.join('..','data\competition_data',x)]
rest_files = {}
for r in restfile:
    key = r.split('\\')[-1].split('.')[0]
    rest_files[key] = pd.read_csv(r ,header=0)


In [3]:
train = pd.merge(all_files['train_set'],all_files['bill_of_materials'], on = 'tube_assembly_id')
test = pd.merge(all_files['test_set'],all_files['bill_of_materials'], on = 'tube_assembly_id')
train = pd.merge(train,all_files['tube'], on = 'tube_assembly_id')
test = pd.merge(test,all_files['tube'], on = 'tube_assembly_id')

train['year'] = train['quote_date'].dt.year
train['month'] = train['quote_date'].dt.month
train['day'] = [date.days for date in train['quote_date'] - dt.date(1982,1,1)]

test['year'] = test['quote_date'].dt.year
test['month'] = test['quote_date'].dt.month
test['day'] = [date.days for date in test['quote_date'] - dt.date(1982,1,1)]


comp_id = ['component_id_'+str(i) for i in range(1,9)]
weight_id = ['weight_id_'+str(i) for i in range(1,9)]

# Add weight to train
for key1 in comp_id:
    i = comp_id.index(key1)
    weight = []
    for key2 in sorted(train[key1].unique(),reverse = True):
        for filename in comp_dict:
            if key2 in comp_dict[filename].index:
                weight.append(comp_dict[filename].loc[key2]['weight'])
    else:
        weight.append(np.nan)
    dfTemp = pd.DataFrame({key1 : sorted(train[key1].unique(),reverse = True),weight_id[i]: weight})
    train = pd.merge(train,dfTemp, how = 'left', on = key1)
train = train[['tube_assembly_id', 'supplier', 'year','month','day', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'cost', 'component_id_1', 'quantity_1','weight_id_1', 'component_id_2', 'quantity_2', 'weight_id_2', 'component_id_3', 'quantity_3', 'weight_id_3', 'component_id_4', 'quantity_4','weight_id_4', 'component_id_5', 'quantity_5', 'weight_id_5', 'component_id_6', 'quantity_6', 'weight_id_6', 'component_id_7', 'quantity_7', 'weight_id_7', 'component_id_8', 'quantity_8', 'weight_id_8', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other']]

# Add weight to test


for key in comp_id:  
    j = comp_id.index(key)
    temp = pd.DataFrame({comp_id[j]: []})
    for filename in comp_dict:
        for key2 in sorted(test[comp_id[j]].unique(),reverse = True):
            i = sorted(test[comp_id[j]].unique(),reverse = True).index(key2)
            if key2 in comp_dict[filename].index:
                temp.loc[i,comp_id[j]] = key2
                temp.loc[i,weight_id[j]] = comp_dict[filename].loc[key2]['weight']
    test = pd.merge(test,temp,how = 'left',on = comp_id[j]) 
test = test[['id', 'tube_assembly_id', 'supplier', 'year','month','day', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'component_id_1', 'quantity_1', 'weight_id_1', 'component_id_2', 'quantity_2', 'weight_id_2', 'component_id_3', 'quantity_3', 'weight_id_3', 'component_id_4', 'quantity_4', 'weight_id_4', 'component_id_5', 'quantity_5', 'weight_id_5', 'component_id_6', 'quantity_6', 'weight_id_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other']]

In [4]:
print list(train.columns)
print
print list(test.columns)

['tube_assembly_id', 'supplier', 'year', 'month', 'day', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'cost', 'component_id_1', 'quantity_1', 'weight_id_1', 'component_id_2', 'quantity_2', 'weight_id_2', 'component_id_3', 'quantity_3', 'weight_id_3', 'component_id_4', 'quantity_4', 'weight_id_4', 'component_id_5', 'quantity_5', 'weight_id_5', 'component_id_6', 'quantity_6', 'weight_id_6', 'component_id_7', 'quantity_7', 'weight_id_7', 'component_id_8', 'quantity_8', 'weight_id_8', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other']

['id', 'tube_assembly_id', 'supplier', 'year', 'month', 'day', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'component_id_1', 'quantity_1', 'weight_id_1', 'component_id_2', 'quantity_2', 'weight_id_2', 'component_id_3', 'quantity_3', 'weight_id_3', 'component_id_4', 'quantity_4', 'weight_i

**Add component columns to each `component_id_*`**

In [5]:
# TRAIN

def comp_column(name):
    if name not in ['component_id_'+str(i) for i in range(1,9)]:
        return "Error"
    else:
        temp = pd.DataFrame({name: []})
        for filename in comp_dict:
            tempdict = {x: [] for x in comp_dict[filename].drop('weight',axis = 1).columns}
            for key in sorted(train[name].unique(),reverse = True):
                i = sorted(train[name].unique(),reverse = True).index(key)
                if key in comp_dict[filename].index:
                    for col in tempdict.keys():
                        temp.loc[i,name] = key
                        temp.loc[i,col] = comp_dict[filename].drop('weight',axis = 1).loc[key][col]
        
        # Dropping any pure NaN columns
        tempcol = temp.columns.copy()
        dropcol = []
        for x in tempcol:
            if sum(pd.isnull(temp[x])) == len(temp[x]):
                dropcol.append(x)
        temp = temp.drop(dropcol,axis=1)
        
        # rename columns for number
        c = name.split('_')[-1]
        temp.columns = [temp.columns[i] + '_comp' + c if i > 0 else temp.columns[i] for i in range(len(temp.columns))]
        return temp
    
def mergeframe(comp_num):
    global train
    
    # Call comp_column()
    comp = 'component_id_'+str(comp_num)
    merge = comp_column(comp)
       
    train = pd.merge(train,merge, how = 'left', on = comp)
    startIndex = list(train.columns).index('weight_id_'+str(comp_num))
    x = list(train.columns.copy())
    newColOrder = []
    for i in range(len(x)):
        if i <= startIndex:
            newColOrder += [x[i]]
        elif i > startIndex and i < startIndex+len(merge.columns):
            newColOrder += [x[-len(merge.columns)+i-startIndex]]
        else:
            newColOrder += [x[i-len(merge.columns)+1]]
    return train[newColOrder]  

# TEST

def comp_columnTest(name):
    if name not in ['component_id_'+str(i) for i in range(1,9)]:
        return "Error"
    else:
        temp = pd.DataFrame({name: []})
        for filename in comp_dict:
            tempdict = {x: [] for x in comp_dict[filename].drop('weight',axis = 1).columns}
            for key in sorted(test[name].unique(),reverse = True):
                i = sorted(test[name].unique(),reverse = True).index(key)
                if key in comp_dict[filename].index:
                    for col in tempdict.keys():
                        temp.loc[i,name] = key
                        temp.loc[i,col] = comp_dict[filename].drop('weight',axis = 1).loc[key][col]
        
        # Dropping any pure NaN columns
        tempcol = temp.columns.copy()
        dropcol = []
        for x in tempcol:
            if sum(pd.isnull(temp[x])) == len(temp[x]):
                dropcol.append(x)
        temp = temp.drop(dropcol,axis=1)
        
        # rename columns for number
        c = name.split('_')[-1]
        temp.columns = [temp.columns[i] + '_comp' + c if i > 0 else temp.columns[i] for i in range(len(temp.columns))]
        return temp
    
def mergeframetest(comp_num):
    global test
    
    # Call comp_column()
    comp = 'component_id_'+str(comp_num)
    merge = comp_columnTest(comp)
       
    test = pd.merge(test,merge, how = 'left', on = comp)
    startIndex = list(test.columns).index('weight_id_'+str(comp_num))
    x = list(test.columns.copy())
    newColOrder = []
    for i in range(len(x)):
        if i <= startIndex:
            newColOrder += [x[i]]
        elif i > startIndex and i < startIndex+len(merge.columns):
            newColOrder += [x[-len(merge.columns)+i-startIndex]]
        else:
            newColOrder += [x[i-len(merge.columns)+1]]
    return test[newColOrder]       
            

            
# comp_column('component_id_7')
#
# comp_columnTest('component_id_6')

In [6]:
for i in range(1,7):
    test = mergeframetest(i)
    
for i in range(1,9):
    train = mergeframe(i)

In [7]:
print '# of train columns:', len(train.columns)
print '# of test columns:',  len(test.columns)

# of train columns: 310
# of test columns: 312


**Checking for full null columns in test**

Any fully null column will be dropped

In [8]:
print 'test'
droptestcol = []
for x in test.columns:
    if sum(pd.isnull(test[x])) == len(test[x]):
        droptestcol += [x]
        print x
print
print 'train'
droptraincol = []
for x in train.columns:
    if sum(pd.isnull(train[x])) == len(train[x]):
        droptraincol += [x]
        print x
test = test.drop(droptestcol, axis = 1)
train = train.drop(droptraincol, axis = 1)
print '# of train columns:', len(train.columns)
print '# of test columns:',  len(test.columns)

test
component_id_7
quantity_7
component_id_8
quantity_8

train
# of train columns: 310
# of test columns: 308


**TUBE_END_FORM** incorporation

In [9]:
def tube_end(col):
    global train
    train2 = train.copy()
    re = rest_files['tube_end_form'].copy()
    tem = []
    if col not in ['end_a','end_x']:
        return 'Error'
    else:
        for i in train2[col]:
            if i == 'NONE':
                tem += [np.nan]
            else:
                t = re[re['end_form_id'] == i]['forming']
                tem += [t.reset_index()['forming'][0]]
    train[col] = tem
    return train

def tube_endTest(col):
    global test
    test2 = test.copy()
    re = rest_files['tube_end_form'].copy()
    tem = []
    if col not in ['end_a','end_x']:
        return 'Error'
    else:
        for i in test2[col]:
            if i == 'NONE':
                tem += [np.nan]
            else:
                t = re[re['end_form_id'] == i]['forming']
                tem += [t.reset_index()['forming'][0]]
    test[col] = tem
    return test


train = tube_end('end_a')
train = tube_end('end_x')

test = tube_endTest('end_a')
test = tube_endTest('end_x')

**TYPE_END_FORM** incorporation

In [10]:
endformtype = train.columns[train.columns.str.contains('end_form')]
endformtypeTest = test.columns[test.columns.str.contains('end_form')]

def type_end(col):
    train2 = train.copy()
    re = rest_files['type_end_form'].copy()
    typeEnd = []
    if col not in endformtype:
        return 'Error'
    else:
        for i in train2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i):
                typeEnd += [np.nan]
            else:
                t = re[re['end_form_id'] == i]['name']
                typeEnd += [t.reset_index()['name'][0]]
    return typeEnd


def type_endTest(col):
    test2 = test.copy()
    re = rest_files['type_end_form'].copy()
    typeEnd = []
    if col not in endformtypeTest:
        return 'Error'
    else:
        for i in test2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i):
                typeEnd += [np.nan]
            else:
                t = re[re['end_form_id'] == i]['name']
                typeEnd += [t.reset_index()['name'][0]]
    return typeEnd


for i in endformtype:
    train[i] = type_end(i)
for i in endformtypeTest:
    test[i] = type_endTest(i)

**TYPE_CONNECTION** incorporation

In [12]:
contype = train.columns[train.columns.str.contains('conn')]
contypeTest = test.columns[test.columns.str.contains('conn')]
def conn_type(col):
    train2 = train.copy()
    re = rest_files['type_connection'].copy()
    conn = []
    if col not in contype:
        return 'Error'
    else:
        for i in train2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i):
                conn += [np.nan]
            else:
                t = re[re['connection_type_id'] == i]['name']
                conn += [t.reset_index()['name'][0]]
    return conn


def conn_typeTest(col):
    test2 = test.copy()
    re = rest_files['type_connection'].copy()
    conn = []
    if col not in contypeTest:
        return 'Error'
    else:
        for i in test2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i):
                conn += [np.nan]
            else:
                t = re[re['connection_type_id'] == i]['name']
                conn += [t.reset_index()['name'][0]]
    return conn

# train3 = train.copy()
# test3 = test.copy()
for i in contype:
    train[i] = conn_type(i)
for i in contypeTest:
    test[i] = conn_typeTest(i)

IndexError: index out of bounds

**TYPE_COMPONENT** incorporation

In [13]:
comptype = train.columns[train.columns.str.contains('component_type')]
comptypeTest = test.columns[test.columns.str.contains('component_type')]
def comp_type(col):
    train2 = train.copy()
    re = rest_files['type_component'].copy()
    comp = []
    if col not in comptype:
        return 'Error'
    else:
        for i in train2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i) or i == 'OTHER':
                comp += [np.nan]
            else:
                t = re[re['component_type_id'] == i]['name']
                comp += [t.reset_index()['name'][0]]
    return comp


def comp_typeTest(col):
    test2 = test.copy()
    re = rest_files['type_component'].copy()
    comp = []
    if col not in comptypeTest:
        return 'Error'
    else:
        for i in test2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i) or i == 'OTHER':
                comp += [np.nan]
            else:
                t = re[re['component_type_id'] == i]['name']
                comp += [t.reset_index()['name'][0]]
    return comp

#train3 = train.copy()
#test3 = test.copy()
for i in comptype:
    train[i] = comp_type(i)
for i in comptypeTest:
    test[i] = comp_typeTest(i)

### Add `specs` data frame

In [135]:
spc = rest_files['specs'].copy()
specstrain = spc[spc['tube_assembly_id'].isin(train['tube_assembly_id'])]
specstest = spc[spc['tube_assembly_id'].isin(test['tube_assembly_id'])]

#train3 = train.copy()
#test3 = test.copy()

train = pd.merge(train,specstrain, how = 'left', on = 'tube_assembly_id')
test = pd.merge(test,specstest, how = 'left', on = 'tube_assembly_id')

## Add component names to test and train

In [169]:
componName = train.columns[train.columns.str.contains('component_id')]
componNameTest = test.columns[test.columns.str.contains('component_id')]

def comp_name(col):
    train2 = train.copy()
    re = rest_files['components'].copy()
    compName = []
    if col not in componName:
        
        return 'Error'
    else:
        for i in train2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i) or i == 'OTHER':
                compName += [np.nan]
            else:
                t = re[re['component_id'] == i]['name']
                compName += [t.reset_index()['name'][0]]
    return compName


def comp_nameTest(col):
    test2 = test.copy()
    re = rest_files['components'].copy()
    compName = []
    if col not in componNameTest:
        return 'Error'
    else:
        for i in test2[col]:
            if i == 'NONE' or i == np.nan or pd.isnull(i) or i == 'OTHER':
                compName += [np.nan]
            else:
                t = re[re['component_id'] == i]['name']
                compName += [t.reset_index()['name'][0]]
    return compName

#train3 = train.copy()
#test3 = test.copy()
for i in componName:
    train[i] = comp_name(i)
for i in componNameTest:
    test[i] = comp_nameTest(i)

In [170]:
train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)


# TESTING BLOCK




# TESTING BLOCK


# TESTING BLOCK


# TESTING BLOCK


# TESTING BLOCK
























# TESTING BLOCK

In [12]:
test[['end_a_1x','end_a_2x','end_x_1x','end_x_2x','end_a','end_x']].head()

,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x
0,N,N,N,N,No,No
1,N,N,N,N,No,No
2,N,N,N,N,No,No
3,N,N,N,N,No,No
4,N,N,N,N,No,No


In [13]:
rest_files.keys()

['tube_end_form',
 'type_connection',
 'components',
 'type_end_form',
 'specs',
 'type_component']

In [142]:
rest_files['components'].tail()

,component_id,name,component_type_id
2043,C-2043,FLANGE,CP-004
2044,C-2044,ADAPTER-INLET,CP-006
2045,C-2045,ADAPTER-INLET,OTHER
2046,C-2046,SLEEVE,OTHER
2047,C-2047,FLANGE,OTHER


### components name testing

In [158]:
compon = rest_files['components'].drop('component_type_id',axis = 1)
compon.columns = ['component_id','comp_id']
#train3 = train.copy()
#test3 = test.copy()

componName = train.columns[train.columns.str.contains('component_id')]
componNameTest = test.columns[test.columns.str.contains('component_id')]
print componName
print componNameTest

Index([u'component_id_1', u'component_id_2', u'component_id_3',
       u'component_id_4', u'component_id_5', u'component_id_6',
       u'component_id_7', u'component_id_8'],
      dtype='object')
Index([u'component_id_1', u'component_id_2', u'component_id_3',
       u'component_id_4', u'component_id_5', u'component_id_6'],
      dtype='object')


In [172]:
train[componName].head(10)

,component_id_1,component_id_2,component_id_3,component_id_4,component_id_5,component_id_6,component_id_7,component_id_8
0,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NUT-FLARED,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
test[componNameTest].head(10)

,component_id_1,component_id_2,component_id_3,component_id_4,component_id_5,component_id_6
0,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
1,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
2,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
3,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
4,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
5,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
6,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
7,NUT-SWIVEL,SLEEVE-ORFS,NaN,NaN,NaN,NaN
8,NUT-FLARED,NaN,NaN,NaN,NaN,NaN
9,NUT-FLARED,NaN,NaN,NaN,NaN,NaN


### `specs` incorporation testing

In [91]:
for i in range(1,11):
    spec = "spec" + str(i)
    print sum(pd.notnull(rest_files['specs'][spec]))

7129
6844
5840
4154
2921
2071
535
106
20
1


In [174]:
test[list(rest_files['specs'].columns)].tail(10)

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10
30225,TA-21182,SP-0024,SP-0063,SP-0070,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN
30226,TA-21182,SP-0024,SP-0063,SP-0070,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN
30227,TA-21182,SP-0024,SP-0063,SP-0070,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN
30228,TA-21187,SP-0004,SP-0024,SP-0069,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN
30229,TA-21188,SP-0007,SP-0024,SP-0070,SP-0080,SP-0082,NaN,NaN,NaN,NaN,NaN
30230,TA-21192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30231,TA-21193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30232,TA-21194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30233,TA-21198,SP-0058,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30234,TA-21199,SP-0058,SP-0080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
train[list(rest_files['specs'].columns)].head(10)

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10
0,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TA-00004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TA-00004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:

specstrain2 = specstrain['tube_assembly_id'].reset_index()['tube_assembly_id']
specstest2 = specstest['tube_assembly_id'].reset_index()['tube_assembly_id']

testing = []
for i in range(len(train['tube_assembly_id'].unique())):
    testing += [specstrain2[i] != train['tube_assembly_id'].unique()[i]]
print 'train subset match error:', sum(testing)

testing = []
for i in range(len(train['tube_assembly_id'].unique())):
    testing += [specstest2[i] != test['tube_assembly_id'].unique()[i]]
print 'test subset match error:', sum(testing)
print
print specstrain.head()
print
print specstest.head()

train subset match error: 0
test subset match error: 0

   tube_assembly_id    spec1    spec2    spec3 spec4 spec5 spec6 spec7 spec8  \
1          TA-00002      NaN      NaN      NaN   NaN   NaN   NaN   NaN   NaN   
3          TA-00004      NaN      NaN      NaN   NaN   NaN   NaN   NaN   NaN   
4          TA-00005      NaN      NaN      NaN   NaN   NaN   NaN   NaN   NaN   
11         TA-00012      NaN      NaN      NaN   NaN   NaN   NaN   NaN   NaN   
12         TA-00013  SP-0004  SP-0069  SP-0080   NaN   NaN   NaN   NaN   NaN   

   spec9 spec10  
1    NaN    NaN  
3    NaN    NaN  
4    NaN    NaN  
11   NaN    NaN  
12   NaN    NaN  

  tube_assembly_id spec1 spec2 spec3 spec4 spec5 spec6 spec7 spec8 spec9  \
0         TA-00001   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2         TA-00003   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5         TA-00006   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
6         TA-00007   NaN   NaN   NaN   NaN   NaN  

### `COMPONENT_TYPE_ID` testing checks

In [51]:
comptype = train.columns[train.columns.str.contains('component_type')]
comptype

Index([u'component_type_id_comp1', u'component_type_id_comp2',
       u'component_type_id_comp3', u'component_type_id_comp4',
       u'component_type_id_comp5', u'component_type_id_comp6'],
      dtype='object')

In [52]:
comptypeTest = test.columns[test.columns.str.contains('component_type')]
comptypeTest

Index([u'component_type_id_comp1', u'component_type_id_comp2',
       u'component_type_id_comp3', u'component_type_id_comp4',
       u'component_type_id_comp5', u'component_type_id_comp6'],
      dtype='object')

In [177]:
'''
for i in comptype:
    print sum(pd.notnull(train[i]))

print 

for i in comptypeTest:
    print sum(pd.notnull(test[i]))
'''    
print train[comptype].head(10)
print test[comptypeTest].head(10)


  component_type_id_comp1 component_type_id_comp2 component_type_id_comp3  \
0        Straight Adapter                     NaN                     NaN   
1        Straight Adapter                     NaN                     NaN   
2        Straight Adapter                     NaN                     NaN   
3        Straight Adapter                     NaN                     NaN   
4        Straight Adapter                     NaN                     NaN   
5        Straight Adapter                     NaN                     NaN   
6        Straight Adapter                     NaN                     NaN   
7        Straight Adapter                     NaN                     NaN   
8        Straight Adapter                     NaN                     NaN   
9        Straight Adapter                     NaN                     NaN   

  component_type_id_comp4 component_type_id_comp5 component_type_id_comp6  
0                     NaN                     NaN                     NaN  


### Checking how many non-null values there are in `contype` and `contypeTest`

In [85]:
'''
for i in range(len(contype)):
    print sum(pd.notnull(train[contype[i]]))

print 

for i in range(len(contypeTest)):
    print sum(pd.notnull(test[contypeTest[i]]))
'''    
print train[contype].head(10)
#print test[contypeTest].tail(10)

  connection_type_id_comp1 connection_type_id_1_comp1  \
0                      NaN  45 deg Inv Flare-SAE J512   
1                      NaN  45 deg Inv Flare-SAE J512   
2                      NaN  45 deg Inv Flare-SAE J512   
3                      NaN  45 deg Inv Flare-SAE J512   
4                      NaN  45 deg Inv Flare-SAE J512   
5                      NaN  45 deg Inv Flare-SAE J512   
6                      NaN  45 deg Inv Flare-SAE J512   
7                      NaN  45 deg Inv Flare-SAE J512   
8                      NaN  45 deg Inv Flare-SAE J512   
9                      NaN  45 deg Inv Flare-SAE J512   

  connection_type_id_2_comp1 connection_type_id_comp2  \
0                      Other                      NaN   
1                      Other                      NaN   
2                      Other                      NaN   
3                      Other                      NaN   
4                      Other                      NaN   
5                      Other  

### Checking how many non-null values there are in `endformtypeTest` and `endformtype`

In [16]:
for i in range(len(endformtypeTest)):
    print sum(pd.notnull(test[endformtypeTest[i]]))


7683
7683
8
948
948
12
5210
5210
18
85
85
5
11
11
6
6


In [17]:
for i in range(len(endformtype)):
    print sum(pd.notnull(train[endformtype[i]]))

7630
7630
1058
1058
16
5320
5320
4
25
49
49
4
5
5
8
8


## Test mergeframe

In [329]:
comp_dict.keys()

['comp_sleeve',
 'comp_nut',
 'comp_adaptor',
 'comp_elbow',
 'comp_other',
 'comp_threaded',
 'comp_hfl',
 'comp_straight',
 'comp_boss',
 'comp_float',
 'comp_tee']

In [313]:
def mergeframeTEST(comp_num):
    train2 = copy.deepcopy(train)
    
    # Call comp_column()
    comp = 'component_id_'+str(comp_num)
    merge = comp_column(comp)
       
    train2 = pd.merge(train2,merge, how = 'left', on = comp)
    startIndex = list(train2.columns).index('weight_id_'+str(comp_num))
    x = list(train2.columns.copy())
    newColOrder = []
    for i in range(len(x)):
        if i <= startIndex:
            newColOrder += [x[i]]
        elif i > startIndex and i < startIndex+len(merge.columns):
            newColOrder += [x[-len(merge.columns)+i-startIndex]]
        else:
            newColOrder += [x[i-len(merge.columns)+1]]
    return newColOrder 

In [305]:
train2 = copy.deepcopy(train)
merge = comp_column('component_id_6')
print len(merge.columns)
train2 = pd.merge(train2,merge, how = 'left', on = 'component_id_6')
t = list(train2.columns).index('weight_id_6')
x = list(train2.columns.copy())
l = []
for i in range(len(x)):
    if i <= t:
        l += [x[i]]
    elif i > t and i < t+len(merge.columns):
        l += [x[-len(merge.columns)+i-t]]
    else:
        l += [x[i-len(merge.columns)+1]]

print len(x)
print len(l)
l, x
# [x[i] if i < t and i > t+len(merge.columns) else x[-1*(len(merge.columns)-i)] for i in range(len(x))]

21
weight_id_6
46
67
67


(['tube_assembly_id',
  'supplier',
  'quote_date',
  'annual_usage',
  'min_order_quantity',
  'bracket_pricing',
  'quantity',
  'cost',
  'component_id_1',
  'quantity_1',
  'weight_id_1',
  'component_id_2',
  'quantity_2',
  'weight_id_2',
  'component_id_3',
  'quantity_3',
  'weight_id_3',
  'component_id_4',
  'quantity_4',
  'weight_id_4',
  'component_id_5',
  'quantity_5',
  'weight_id_5',
  'component_id_6',
  'quantity_6',
  'weight_id_6',
  'part_name_comp6',
  'orientation_comp6',
  'nominal_size_2_comp6',
  'adaptor_angle_comp6',
  'component_type_id_comp6',
  'connection_type_id_1_comp6',
  'thread_size_1_comp6',
  'end_form_id_1_comp6',
  'end_form_id_2_comp6',
  'unique_feature_comp6',
  'length_1_comp6',
  'length_2_comp6',
  'thread_pitch_1_comp6',
  'height_over_tube_comp6',
  'groove_comp6',
  'connection_type_id_comp6',
  'base_type_comp6',
  'outside_shape_comp6',
  'type_comp6',
  'base_diameter_comp6',
  'component_id_7',
  'quantity_7',
  'weight_id_7',
  'c

### Examining the merge process for different `component_id_*`

I explored `component_id_6,7,8` and they seem to work well

In [253]:
train2 = copy.deepcopy(train)
print len(train2.iloc[:,0])
train2 = pd.merge(train2,comp_column('component_id_7'), how = 'left', on = 'component_id_7')
print train2[pd.notnull(train['component_id_7'])].iloc[:,-10:]
print len(train2.iloc[:,0])

30213
      end_a_1x end_a_2x end_x_1x end_x_2x   end_a   end_x  num_boss  \
6179         N        N        N        N  EF-021  EF-021         0   
6180         N        N        N        N  EF-021  EF-021         0   
6181         N        N        N        N  EF-021  EF-021         0   
6533         N        N        Y        Y  EF-023  EF-023         0   
17706        N        N        Y        Y  EF-023  EF-023         1   
17707        N        N        Y        Y  EF-023  EF-023         1   
17708        N        N        Y        Y  EF-023  EF-023         1   
18896        N        Y        N        Y  EF-023  EF-023         0   

       num_bracket  other part_name_c7  
6179             0      0        PLATE  
6180             0      0        PLATE  
6181             0      0        PLATE  
6533             0      6        PLATE  
17706            0      2        PLATE  
17707            0      2        PLATE  
17708            0      2        PLATE  
18896            0      8 

### Testing a function to see how each `component_id_*` gets a data frame associated with it

Looking to build a function `comp_column(name)` that will take care of this for me. This is my scratch work for testing the function and seeing if it is working properly. Combining dictionaries didn't work well because the alignment between different columns is off - especially when in `component_id_6` C-2005 doesn't have a `part_name`, but it gets assigned one when concatenating the dictionaries together, which is bad. So I opted for a bruteforce approach of building each row one by one (3 nested for-loops, ugly).

In [250]:
test = pd.DataFrame({'component_id_6': []})
print sorted(train['component_id_6'].unique(),reverse = True)
for filename in comp_dict:
    tempdict = {x: [] for x in comp_dict[filename].drop('weight',axis = 1).columns}
    for key in sorted(train['component_id_6'].unique(),reverse = True):
        i = sorted(train['component_id_6'].unique(),reverse = True).index(key)
        if key in comp_dict[filename].index:
            for col in tempdict.keys():
                # test[i,col] = comp_dict[filename].drop('weight',axis = 1).loc[key][col]
                # tempdict[col].append(comp_dict[filename].drop('weight',axis = 1).loc[key][col])
                test.loc[i,'component_id_6'] = key
                test.loc[i,col] = comp_dict[filename].drop('weight',axis = 1).loc[key][col]
                print key, col, comp_dict[filename].drop('weight',axis = 1).loc[key][col]
    #if len(tempdict[tempdict.keys()[0]])>0:
        #tempframe = pd.DataFrame(tempdict)   
        #test = pd.concat([test,tempframe],axis=1)
                # .append(comp_dict[filename].loc[key][col])
# train2 = pd.DataFrame({key1 : sorted(train[key1].unique(),reverse = True),weight_id[i]: weight})
# print tempdict
# print test
tempcol = test.columns.copy()
drop = []
for x in tempcol:
    if sum(pd.isnull(test[x])) == len(test[x]):
        drop.append(x)
test = test.drop(drop,axis=1)
c = 'component_id_6'.split('_')[-1]
test.columns = [test.columns[i] + '_c' + c if i > 0 else test.columns[i] for i in range(len(test.columns))]
print test.iloc[:,:10]
print test.iloc[:,10:]

['C-2005', 'C-1920', 'C-1560', 'C-0981', 'C-0967', 'C-0965', 'C-0934', 'C-0663', 'C-0378', 'C-0353', nan]
C-1920 part_name PLATE
C-1560 part_name FLANGE
C-0981 part_name PLATE
C-0967 part_name BRACKET
C-0934 part_name PLATE
C-0663 part_name TUBE
C-0378 part_name VALVE BODY
C-0353 part_name PIPE
C-2005 orientation Yes
C-2005 nominal_size_3 nan
C-2005 nominal_size_2 19.05
C-2005 nominal_size_1 nan
C-2005 nominal_size_4 nan
C-2005 adaptor_angle 90.0
C-2005 component_type_id CP-015
C-2005 hex_size nan
C-2005 thread_size_4 nan
C-2005 connection_type_id_1 B-002
C-2005 connection_type_id_2 nan
C-2005 connection_type_id_3 nan
C-2005 connection_type_id_4 nan
C-2005 thread_size_1 1.187
C-2005 thread_size_2 nan
C-2005 thread_size_3 nan
C-2005 end_form_id_4 nan
C-2005 end_form_id_1 A-001
C-2005 end_form_id_2 A-004
C-2005 end_form_id_3 nan
C-2005 overall_length nan
C-2005 unique_feature Yes
C-2005 length_1 37.3
C-2005 length_3 nan
C-2005 length_2 37.3
C-2005 length_4 nan
C-2005 thread_pitch_4 nan
C

### testing if the merge works properly

testing this script for adding the `weight` column

In [100]:
i = comp_id.index('component_id_6')
print sorted(train['component_id_6'].unique(), reverse = True)
weight = []
for key2 in sorted(train['component_id_6'].unique(), reverse = True):
    for filename in comp_dict:
        if key2 in comp_dict[filename].index:
            weight.append(comp_dict[filename].loc[key2]['weight'])
            print filename, key2, comp_dict[filename].loc[key2]['weight']
            print 
else:
    weight.append(np.nan)
train3 = pd.DataFrame({'component_id_6': sorted(train['component_id_6'].unique(), reverse = True),
                       'weight': weight})
train3.head(20)


['C-2005', 'C-1920', 'C-1560', 'C-0981', 'C-0967', 'C-0965', 'C-0934', 'C-0663', 'C-0378', 'C-0353', nan]
comp_threaded C-2005 0.326

comp_other C-1920 0.15

comp_other C-1560 6.9

comp_other C-0981 0.524

comp_other C-0967 1.138

comp_boss C-0965 0.5

comp_other C-0934 0.701

comp_other C-0663 3.771

comp_other C-0378 0.094

comp_other C-0353 0.326



,component_id_6,weight
0,C-2005,0.326
1,C-1920,0.150
2,C-1560,6.900
3,C-0981,0.524
4,C-0967,1.138
5,C-0965,0.500
6,C-0934,0.701
7,C-0663,3.771
8,C-0378,0.094
9,C-0353,0.326


In [438]:
# testing on test

test2 = test.copy()
print test2.columns
comp_id = ['component_id_'+str(i) for i in range(1,9)]
weight_id = ['weight_id_'+str(i) for i in range(1,9)]
temp = pd.DataFrame({'component_id_6': []})
for filename in comp_dict:
    for key2 in sorted(test2['component_id_6'].unique(),reverse = True):
        i = sorted(test2['component_id_6'].unique(),reverse = True).index(key2)
        if key2 in comp_dict[filename].index:
            print filename, key2
            temp.loc[i,'component_id_6'] = key2
            temp.loc[i,weight_id[5]] = comp_dict[filename].loc[key2]['weight']
test2 = pd.merge(test2,temp,how = 'left',on = 'component_id_6')

print test2.columns
print temp

Index([u'id', u'tube_assembly_id', u'supplier', u'quote_date', u'annual_usage',
       u'min_order_quantity', u'bracket_pricing', u'quantity',
       u'component_id_1', u'quantity_1', u'component_id_2', u'quantity_2',
       u'component_id_3', u'quantity_3', u'component_id_4', u'quantity_4',
       u'component_id_5', u'quantity_5', u'component_id_6', u'quantity_6',
       u'component_id_7', u'quantity_7', u'component_id_8', u'quantity_8',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other'],
      dtype='object')
comp_other C-1049
comp_other C-0557
comp_other C-0279
comp_other C-0176
comp_threaded C-1639
comp_straight C-2021
comp_boss C-1065
comp_boss C-0965
Index([u'id', u'tube_assembly_id', u'supplier', u'quote_date', u'annual_usage',
       u'min_order_quantity', u'bracket_pricing', u'quantity',
       u'component_id_1', u'quan

### Examining Jingyi's files

In [22]:
training1 = pd.read_csv("../train.csv", index_col = 0, header = 0)
trainning2 = pd.read_csv("../train4.csv", index_col = 0, header = 0)
print training2.columns

Index([u'tube_assembly_id', u'supplier', u'bracket_pricing', u'quantity',
       u'cost', u'component_id_1_name', u'component_1_weight', u'quantity_1',
       u'component_id_2_name', u'component_2_weight', u'quantity_2',
       u'component_id_3_name', u'component_3_weight', u'quantity_3',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_2x', u'end_x_2x', u'end_a', u'end_x', u'year',
       u'month', u'day'],
      dtype='object')
